In [1]:
from create_field import *
from field import *

import numpy as np
from matplotlib import pyplot as plt
import cv2
import json
import os

### Функция запуска

Обязательный параметр - путь до конфига

Также можно передать следующие параметры:

1. Ключ для сохранения статистики без самих картинок поля (установить = False для сохранения картинок)
2. Путь до директории сохранения результатов

In [2]:
def run_field(cfg_path, save_only_stat=True, save_direcory='results'):

    with open(os.path.join('configs', 'cfg_vis.json'), 'r') as f:
        cfg_vis = json.load(f)

    cell_size = cfg_vis['cell_size']
    window_name = cfg_vis['window_name']
    show = cfg_vis['show']
    
    with open(cfg_path, 'r') as f:
        config = json.load(f)
    
    max_iterations = config['iterations']
    neighbours_type = config['field_type']
    field_size = config['field_size']
    grid = np.array(json.loads(config['field']))
    opinion_matrix = np.array(json.loads(config['opinions']))
    
    if neighbours_type == 'Ring':
        life1 = Ring(
            field=grid,
            shape=grid.shape,
            opinion_matrix=opinion_matrix
            )
    else:
        life1 = Cross( 
            field=grid,
            shape=grid.shape,
            opinion_matrix=opinion_matrix
            )
        
    result_imgs = []
    result_stat = []
    result_entropy = []

    for _ in range(max_iterations):
        img = life1.plot(cell_size)
        result_imgs.append(img)
        result_stat.append(life1.calculate_stat())
        result_entropy.append(life1.calculate_entropy())

        if show:
            cv2.namedWindow(window_name)
            cv2.imshow(window_name, img)
            key = cv2.waitKey(0)
            if key == 27:
                break

        life1.life_cycle()
            
    cv2.destroyAllWindows()
    
    
    if not os.path.exists(save_direcory):
        os.makedirs(save_direcory)

    if not save_only_stat:
        imgs_path = os.path.join(save_direcory, 'imgs')

        if not os.path.exists(imgs_path):
            os.makedirs(imgs_path)

        for i, img in enumerate(result_imgs):
            if i % 10 == 0:
                cv2.imwrite(
                    os.path.join(imgs_path, f'{i}.png'),
                    img
                    )

    with open(os.path.join(save_direcory, 'stat.json'), 'w') as f:
        json.dump(result_stat, f)
    
    plt.plot(range(len(result_stat)), result_stat)
    plt.title('Доля нулевого мнения')
    plt.savefig(os.path.join(save_direcory, 'plot_ratio.png'))
    plt.clf()

    plt.plot(range(len(result_entropy)), result_entropy)
    plt.title('Энтропия')
    plt.savefig(os.path.join(save_direcory, 'plot_entropy.png'))
    plt.clf()
    
    return result_imgs, result_stat


### выбор конфига и запуск программы

In [3]:
cfg_path = os.path.join('configs', 'config_2', 'config.json')
save_path = os.path.join('results', 'config_2')

r_i, r_s = run_field(cfg_path, save_only_stat=False, save_direcory=save_path)

<Figure size 640x480 with 0 Axes>